## Simple QA Telegram chat bot 

* Author: Boris Ustyugov
* This is my solution to final assignment for the offline course 'Advanced Data Science with Sberbank'  https://moscoding.ru/sber-data-science/
* Template code was provided in course materials, lecture 7 https://github.com/gaphex/mcs-nlp/ (Author: Denis Antyuhov)
* Data was provided by some bank


### In this assignment the goal is to process a large corpus of unstructured chat data, discover it's structure and build a Question Answering system based on it

In [10]:
import numpy as np
import json

import nltk
from gensim.models import Word2Vec
import pymystem3

from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_similarity

# telegram bot 
import logging
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters

###  Unzip and explore the data

In [2]:
# !unzip ./support_logs.zip

In [3]:
faq = open("./support_faq.txt").read()

In [4]:
raw_chats = open("./support_chats.txt").read()

In [5]:
print(faq[:995])

1 . Могу ли я поменять ПИН-код своей карты <bankname> Банка в банкомате ?
Да , если банкомат поддерживает данную функцию .

2 . В каких валютах можно расплачиваться картой <bankname> Банка ?
Расплачиваться картой и снимать наличные можно в любой валюте .

3 . При оплате товаров за границей , я оплачиваю НДС в казну другого государства , возможно ли вернуть эти средства ?
Да , возможно . Для этого вам необходимо в магазине , в котором вы приобретаете товар , попросить предоставить дополнительный чек Tax Free . На данный чек необходимо поставить штамп на таможне . Возврат средств по чекам Tax Free возможен в международном аэропорту ( в стране , где совершалась покупка ) в специальном пункте возврата НДС .

4 . Безопасно ли направлять ссылку на скачивание выписки на сайте банка ? Может ли другой человек получить мою выписку на сайте банка ?
Получать выписку по ссылке в письме так же безопасно , как получать выписку в виде вложения - ссылка уникальна и генерируется случайным образом .


In [6]:
print(raw_chats[:1000])

Chat number = 229391
Client Id = 4569861

20:31:24 Клиент : Добрый день ! Могу ли я частично изъять со счета денежные средства ранее 1 месяцев ?
20:32:33 Сотрудник : Здравствуйте !
20:33:15 Сотрудник : Только полностью закрыть вклад .
20:33:34 Клиент : Сколько будет стоить перевод со своей карты в размере 111 т.р . на две карты клиентам <bankname> банк ?
-- -- -- --

Chat number = 229390
Client Id = 1522643

20:30:51 Клиент : Здравствуйте !
20:31:38 Клиент : Подскажите , где в онлайн-банке партнерские спецпредложения по картам ?
20:32:15 Сотрудник : Здравствуйте !
20:32:58 Сотрудник : К сожалению , в новой версии этого раздела еще нет .
20:33:20 Клиент : Ясно , будем ждать . Всего доброго !
20:33:26 Сотрудник : Всего Вам доброго ! Будут еще вопросы , обращайтесь !
-- -- -- --

Chat number = 229389
Client Id = 4680324

20:30:46 Клиент : Добрый вечер
20:31:23 Сотрудник : Здравствуйте !
20:32:04 Сотрудник : Чем я могу Вам помочь ?
20:32:46 Клиент : У меня была операция по привязке карты к

### The data consists of 2 files. The first one contains raw logs of some bank's customer support. The second one contains frequently asked questions with answers. 

### A good place to start would be to turn the FAQ into a more structured form, then find paraphrases for each question from the chat logs.

In [7]:
faq_qna = []
buf = []
for line in faq.split("\n"):
    if line == '' and len(buf):
        faq_qna.append(buf)
        buf = []
    else:
        buf.append(line)
        
FAQ = []
for faq_entry in faq_qna:
    faq_dict = {}
    faq_dict['answer'] = faq_entry[1]
    faq_dict['question'] = faq_entry[0]
    faq_dict['paraphrased_questions'] = [faq_entry[0]] ## add paraphrases
    FAQ.append(faq_dict)

In [8]:
FAQ

[{'answer': 'Да , если банкомат поддерживает данную функцию .',
  'paraphrased_questions': ['1 . Могу ли я поменять ПИН-код своей карты <bankname> Банка в банкомате ?'],
  'question': '1 . Могу ли я поменять ПИН-код своей карты <bankname> Банка в банкомате ?'},
 {'answer': 'Расплачиваться картой и снимать наличные можно в любой валюте .',
  'paraphrased_questions': ['2 . В каких валютах можно расплачиваться картой <bankname> Банка ?'],
  'question': '2 . В каких валютах можно расплачиваться картой <bankname> Банка ?'},
 {'answer': 'Да , возможно . Для этого вам необходимо в магазине , в котором вы приобретаете товар , попросить предоставить дополнительный чек Tax Free . На данный чек необходимо поставить штамп на таможне . Возврат средств по чекам Tax Free возможен в международном аэропорту ( в стране , где совершалась покупка ) в специальном пункте возврата НДС .',
  'paraphrased_questions': ['3 . При оплате товаров за границей , я оплачиваю НДС в казну другого государства , возможно 

In [11]:
with open('FAQ_PROJECT_DATA.json', 'w') as f:
    json.dump(FAQ, f)

### Build  word2vec model from raw chat 

In [12]:
# merge all phrases 
all_texts = [s.split('Клиент :')[1] for s in raw_chats.split('\n') if ('Клиент :' in s)] + [s.split('Сотрудник :')[1] for s in raw_chats.split('\n') if ('Сотрудник :' in s)] + [d['question'] for d in FAQ] + [d['answer'] for d in FAQ]
len(all_texts)

549509

In [13]:
def my_tokenizer(text):
    return nltk.regexp_tokenize(text, '\w+')

In [15]:
%%time
texts_tokenized = [my_tokenizer(text.lower()) for text in all_texts]

CPU times: user 7.59 s, sys: 225 ms, total: 7.82 s
Wall time: 7.86 s


In [16]:
%%time
w2v_model = Word2Vec(texts_tokenized, size=300, window=5, min_count=5)

CPU times: user 2min 23s, sys: 414 ms, total: 2min 24s
Wall time: 52 s


In [17]:
w2v_model.wv.most_similar('банкомат')

[('терминал', 0.7359819412231445),
 ('банкоматы', 0.6131069660186768),
 ('кассу', 0.6120798587799072),
 ('мкб', 0.6066728830337524),
 ('терминалы', 0.5982102155685425),
 ('евросеть', 0.5974043011665344),
 ('систему', 0.5877324342727661),
 ('сервис', 0.5875207185745239),
 ('сбербанк', 0.565697431564331),
 ('сбер', 0.5596617460250854)]

### Build  evaluation functions

In [18]:
def calc_score_for_engine(engine, val_json_path):
    data = []
    with open(val_json_path, 'r') as f:
        for q in json.load(f):
            queries = q['paraphrased_questions']
            answer = q['answer']           
     
            results = [engine.get_top(query, top_k=3) for query in queries]
            for r in results:
                data.append([answer] + r)
            
    ra1 = calc_recall(data, 1)
    ra3 = calc_recall(data, 3)
    print("recall @1: {}\nrecall @3: {}".format(ra1, ra3))
    return ra1, ra3
            
def calc_recall(data, k, bootstrap=0, subsample_rate=None):
    """
    :param data: 2d matrix
    data[i, 0] - true answer, data[i, 1:] - predicted answers, sorted by decreasing score.
    """
    count = np.zeros(1 + bootstrap)
    count_hit = np.zeros(1 + bootstrap)
    for fields in data:
        query = fields[0]

        if subsample_rate is None:
            increment = np.random.poisson(lam=1, size=bootstrap)
        else:
            increment = np.random.binomial(1, subsample_rate, bootstrap)
        increment = np.hstack([[1], increment])

        if query in fields[1:k+1]:
            count_hit += increment
        count += increment

    recall = count_hit / count

    return recall[0]

### Build IR engine

In [19]:
# bag of words encoder
def bow_encoder(wmodel, tokenizer, text, vsize=300):
    """
    This function encodes text into a vector.
    
    First, it tokenizes input text using the provided tokenizer function.
    Then it uses the provided word2vec model to get the vectors corresponding to text's tokens.
    Finally, it computes an average of all token's vectors and returns it.
    
    If the function failed to find and encode any words, it should at least return a vector of zeros.
    """
    tokens = tokenizer(text)
    
    zero_vector = np.zeros(vsize)
    word_vectors = []
    
    for token in tokens:
        if token in wmodel:
            word_vectors.append(wmodel[token])
            
    if len(word_vectors):
        sent_vector = np.mean(word_vectors, axis=0)
    else:
        sent_vector = zero_vector

    return sent_vector

In [21]:
class ENGINE_3(object):
    def __init__(self, kbase_path, w2v_model):
        with open(kbase_path, 'r') as f:
            self.knowledge_base = json.load(f)
        self.lemmatizer = pymystem3.Mystem()
        self.w2v_model = w2v_model
        
        self.answers = np.array([t['answer'] for t in self.knowledge_base])
        
        self.vectorized_kbase, self.class_indexes = self.vectorize_knowledge_base()
    
    def vectorize(self, data):
        """
        Turns a list of N strings into their vector representation using self.w2v_model.
        In the simplest case, averages the word vectors of all words in a sentence.
        Returns a a matrix of shape [N, 300]
        """
        vectorized = []
        
        for d in data:
            vectorized.append(bow_encoder(self.w2v_model, self.tokenize_and_lemmatize, d))
        
 
        return np.array(vectorized)
        
    def vectorize_knowledge_base(self):
        """
        Vectorizes all questions AND paraphrased questions using the vectorize function.
        Builds a list containing class id (it's index in self.knowledge_base) for each vectorized question.
        
        Example: you vectorized 1 question and 5 paraphrases for that question
        Then you should append the ID of the question to class_labels list 5+1=6 times.
        """
        vectors = []
        class_labels = []
        
        for i , t in enumerate(self.knowledge_base):
            vc = np.vstack([self.vectorize([t['question']]),
                           self.vectorize(t['paraphrased_questions']) ])
            vectors.append(vc)
            class_labels.append(i)
            class_labels += [i]*len(t['paraphrased_questions'])
        
        
        return np.vstack(vectors), class_labels
    
    def compute_class_scores(self, similarities):
        """
        Accepts an array of similarities of shape (self.class_indexes, )
        Computes scores for classes.
        Returns a dictionary of size (n_classes) that looks like
        {
            0: 0.3,
            1: 0.1,
            2: 0.0,
            class_n_id: class_n_score
            ...
        }
        """       
 
        class_scores = dict(zip(range(len(self.answers)), [0]*len(self.answers)))
        for ci, sc in zip(self.class_indexes, similarities):
            class_scores[ci] += sc
        
        return class_scores
        
    def tokenize_and_lemmatize(self, text):
        analysis = self.lemmatizer.analyze(text.strip())
        tokens = []
        for an in analysis:
            if 'analysis' in an:
                try:
                    tokens.append(an['analysis'][0]['lex'])
                except IndexError:
                    tokens.append(an['text'])
        return tokens
    
    def get_top(self, query, top_k=3):
        if isinstance(query, str):
            query = [query]
            
        vectorized_query = self.vectorize(query)
        css = cosine_similarity(vectorized_query, self.vectorized_kbase)[0]
        scores = self.compute_class_scores(css)
        
        sorted_scores = sorted(scores.items(), key= lambda x: x[1])[::-1][:top_k]
        top_classes = np.array([c[0] for c in sorted_scores])
        top_answers = list(self.answers[top_classes])

        return top_answers    

### Add paraphrases for FAQ from raw chat

In [22]:
# filter out queries from chat data
chat_queries = [s.split('Клиент :')[1] for s in raw_chats.split('\n') if ('Клиент :' in s)]
chat_queries = [s for s in chat_queries if len(s)>50]
chat_queries = [s for s in chat_queries if s[-1]=='?']
print(len(chat_queries))

30646


In [23]:
chat_queries[:10]

[' Добрый день ! Могу ли я частично изъять со счета денежные средства ранее 1 месяцев ?',
 ' Сколько будет стоить перевод со своей карты в размере 111 т.р . на две карты клиентам <bankname> банк ?',
 ' Подскажите , где в онлайн-банке партнерские спецпредложения по картам ?',
 ' Будет ли взыматься комиссия если я переведу другу деньги по номеру его карты , в сторонний банк ?',
 ' здравствуйте , если я оплачу через приложение коммунальные платежи как мне получить чек , что я оплатила ?',
 ' У меня при себе нет логина и пароля . я пользуюсь только мобильным приложением . а вы не можете внести эти данные в систему ?',
 ' здравствуйте , скажите пожалуйста , я случайно перевёл деньги не на тот счет телефона ( оплатил мобильную связь ) можно ли как то вернуть деньги ?',
 ' Добрый вечер ! Подскажите , действует ли ***** **** при оплате через мобильный банк ?',
 ' Добрый вечер ! действует ли ***** **** при оплате через мобильный банк ?',
 ' Каким образом я могу досрочно погасить кредит и как эт

In [24]:
DATA_PATH = './FAQ_PROJECT_DATA.json'

engine = ENGINE_3(DATA_PATH, w2v_model)

In [25]:
%%time
# vectorize 
base_queries = [d['question'] for d in FAQ]
base_queries_vec = engine.vectorize(base_queries)
chat_queries_vec = engine.vectorize(chat_queries)

CPU times: user 15.5 s, sys: 1.12 s, total: 16.6 s
Wall time: 58 s


In [26]:
%%time
# fit NearestNeighbors model
NN_model = NearestNeighbors(metric='cosine')
NN_model.fit(chat_queries_vec)

CPU times: user 13 ms, sys: 1 ms, total: 14 ms
Wall time: 12.8 ms


In [27]:
%%time
# add nearest paraphrases from chat queries
K_PARAPH = 5
for i in range(len(FAQ)):
    base_query = base_queries_vec[i,:].reshape((1,-1))
    neigh_ids = list(NN_model.kneighbors(base_query, K_PARAPH, return_distance=False)[0])
    FAQ[i]['paraphrased_questions'] = [chat_queries[i] for i in neigh_ids]

CPU times: user 55.7 s, sys: 5.62 s, total: 1min 1s
Wall time: 31.1 s


In [28]:
FAQ

[{'answer': 'Да , если банкомат поддерживает данную функцию .',
  'paraphrased_questions': [' Хотел узнать , могу ли я сам изменить пин код дебетовой карты , например в банкомате ?',
   ' Здравствуйте ! Могу ли я установить на свою карту пин код ?',
   ' Здравствуйте ! Могу ли я установить на свою карту пин код ?',
   ' могу я изменить пин код на своей карте <bankname> Блэк ?',
   ' Добрый вечер , могу ли я поменять пин код в онлайн банке ?'],
  'question': '1 . Могу ли я поменять ПИН-код своей карты <bankname> Банка в банкомате ?'},
 {'answer': 'Расплачиваться картой и снимать наличные можно в любой валюте .',
  'paraphrased_questions': [' Здравствуйте ! Какие дебетовые карты можно открыть в валюте ? Только T****** B**** ?',
   ' где в клиент банке под андроид можно открыть виртуальную карту ?',
   ' Ясно . Каким образом в дальнейшем можно обналичить валютную карту ?',
   ' Можно ли держателю дополнительной карте обеспечить доступ в интернет-банк и мобильный банк ?',
   ' И какую макс

In [29]:
with open('FAQ_PROJECT_DATA.json', 'w') as f:
    json.dump(FAQ, f)

### Run Engine

In [30]:
DATA_PATH = './FAQ_PROJECT_DATA.json'

engine = ENGINE_3(DATA_PATH, w2v_model)

In [31]:
# recall scores
r1, r3 = calc_score_for_engine(engine, DATA_PATH)

recall @1: 0.8514619883040936
recall @3: 0.9812865497076023


### Run Telegram bot

!!! Before starting the bot you should get a TOKEN and put it in config.py

How to get a new token:
* find @BotFather
* write /newbot
* write in bot_name and user_name
* get a token for the new bot

In [32]:
from config import TOKEN, LOG_FILE

# Enable logging
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
# Logging to file
fh = logging.FileHandler(LOG_FILE)
fh.setLevel(logging.DEBUG)
fh.setFormatter(formatter)
# Logging to console
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
ch.setFormatter(formatter)

logger.addHandler(fh)
logger.addHandler(ch)


class Bot:
    def __init__(self):

        self.updater = Updater(TOKEN)
        self.dsp = self.updater.dispatcher

        # register handler functions which define how the bot reacts to events
        self.dsp.add_handler(CommandHandler("start", get_help))
        self.dsp.add_handler(CommandHandler("help", get_help))
        self.dsp.add_handler(CommandHandler("sentiment", get_sentiment))
        self.dsp.add_handler(CommandHandler("answer", get_answer))
        self.dsp.add_handler(MessageHandler(Filters.text, echo))
        self.dsp.add_error_handler(error)

        logger.info('Im alive!')

    def power_on(self):
        # start the Bot
        self.updater.start_polling()
        self.updater.idle()

# define command handlers. These usually take the two arguments: bot and
# update. Error handlers also receive the raised TelegramError object in error.


def echo(bot, update):
    logger.info('echo recieved message: {}'.format(update.message.text))
    bot.sendMessage(update.message.chat_id, text=update.message.text)


def error(bot, update, error):
    # all uncaught telegram-related exceptions will be rerouted here
    logger.error('Update "%s" caused error "%s"' % (update, error))


def get_help(bot, update):
    logger.info('get_help recieved message: {}'.format(update.message.text))
    help_msg = ('Greetings, {} {}! Name is {}, at your service.\n'
                'I currently support the following commands:\n'
                '/start - begins our chat and prints this message\n'
                '/help - prints this message\n'
                '/sentiment [message] - predicts the sentiment of the message\n'
                '/answer [question] - answers the question').format(
        update.message.from_user.first_name, update.message.from_user.last_name, bot.name)
    bot.sendMessage(update.message.chat_id, text=help_msg)


def get_sentiment(bot, update):
    '''
    Now determine the sentiment of usr_msg.
    This should return a real number in [0,1].
    '''
    logger.info('get_sentiment recieved message: {}'.format(update.message.text))
    try:
        # get message text without the command '/sentiment'
        usr_msg = update.message.text.split(' ', maxsplit=1)[1]
        msg_sentiment = 0.5

        bot.sendMessage(update.message.chat_id, text=msg_sentiment)
    except IndexError:
        bot.sendMessage(update.message.chat_id, text='Write your message after the command')
    except Exception as e:
        logger.error(e)
        
def get_answer(bot, update):
    '''
    Now determine answer based on IR engine.
    This should return an answer from FAQ'
    '''
    logger.info('get_answer recieved message: {}'.format(update.message.text))
    try:
        # get message text without the command 
        usr_msg = update.message.text

        answer = engine.get_top(usr_msg, top_k=3)[0]
                        
        bot.sendMessage(update.message.chat_id, text=answer)
    except IndexError:
        bot.sendMessage(update.message.chat_id, text='Write your message after the command')
    except Exception as e:
        logger.error(e)

my_bot = Bot()
my_bot.power_on()

2017-12-11 17:20:07,467 - __main__ - INFO - Im alive!
2017-12-11 17:20:19,773 - __main__ - INFO - get_answer recieved message: /answer Где можно заплатить по кредит
2017-12-11 17:20:47,452 - __main__ - INFO - get_answer recieved message: /answer Зачем мне вообще кредитная карта
2017-12-11 17:21:06,931 - __main__ - INFO - get_answer recieved message: /answer Какой тип счета выбирать в банкомате при снятии наличных
